# Text Classification (Amazon Ml challenge 2021)

Importing necessary libraries

In [1]:
import statistics as sta
import seaborn as sns
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [3]:
import csv

Unzipping and getting the data

In [4]:
!wget https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip

--2021-08-08 13:29:23--  https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.40.5
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.40.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1061576029 (1012M) [binary/octet-stream]
Saving to: ‘dataset52a7b21.zip’

dataset52a7b21.zip  100%[===================>]   1012M  17.3MB/s    in 60s     

2021-08-08 13:30:24 (16.8 MB/s) - ‘dataset52a7b21.zip’ saved [1061576029/1061576029]



In [5]:
!unzip dataset52a7b21.zip

Archive:  dataset52a7b21.zip
   creating: dataset/
  inflating: dataset/train.csv       
  inflating: dataset/sample_submission.csv  
  inflating: dataset/test.csv        
  inflating: dataset/.~lock.train.csv#  


In [7]:
test = pd.read_csv('test.csv' ,escapechar="\\", quoting=csv.QUOTE_NONE)

In [8]:
train = pd.read_csv('train.csv' ,escapechar="\\", quoting=csv.QUOTE_NONE)

Understanding the data
  

In [9]:
test.head()

,PRODUCT_ID,TITLE,DESCRIPTION,BULLET_POINTS,BRAND
0,1,"Command 3M Small Kitchen Hooks, White, Decorat...",Sale Unit: PACK,[INCLUDES - 9 hooks and 12 small indoor strips...,Command
1,2,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...,Synthetic leather palm with double-layer thumb...,[Silicone printing for a better grip. Long las...,O'Neal
2,3,"NFL Detroit Lions Portable Party Fridge, 15.8 ...",Boelter Brands lets you celebrate your favorit...,[Runs on 12 Volt DC Power or 110 Volt AC Power...,Boelter Brands
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...,Features: 50 Station Phonebook Corded Phone Al...,Panasonic Landline Phones doesn't come with a ...,Panasonic
4,5,Zero Baby Girl's 100% Cotton Innerwear Bloomer...,"Zero Baby Girl Panties Set. 100% Cotton, Breat...","[Zero Baby Girl Panties, Pack of 6, 100% Cotto...",Zero


In [10]:
train.head()

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,NaN,NaN,NaN,4


In [11]:
train.shape

(2903024, 5)

In [12]:
train.dropna(inplace = True)

In [13]:
train.shape

(2110586, 5)

In [14]:
test.dropna(inplace = True)

In [15]:
test.shape

(97793, 5)

In [16]:
np.random.seed(75)

Getting a small amount of data to make training easier

In [17]:
small_train = train.head(30000)

In [18]:
df = small_train[['BROWSE_NODE_ID','TITLE']].copy()

In [19]:
df.head()

,BROWSE_NODE_ID,TITLE
0,0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch"
1,1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ..."
5,5,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...
6,6,Glance Women's Wallet (Black) (LW-21)
7,7,Wild Animals Hungry Brain Educational Flash Ca...


In [20]:
df.shape

(30000, 2)

In [21]:
df['TITLE'] = [entry.lower() for entry in df['TITLE']]

In [22]:
df.head()

,BROWSE_NODE_ID,TITLE
0,0,"pete the cat bedtime blues doll, 14.5 inch"
1,1,"the new yorker nyhm014 refrigerator magnet, 2 ..."
5,5,men's full sleeve raglan t-shirts denim t-shir...
6,6,glance women's wallet (black) (lw-21)
7,7,wild animals hungry brain educational flash ca...


Splitting the data into train and test dataframes

In [23]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['TITLE'],
                                                                    df['BROWSE_NODE_ID'],test_size=0.2)


In [24]:
Train_Y=Train_Y.to_numpy()

In [25]:
Test_Y=Test_Y.to_numpy()

In [26]:
Train_X=Train_X.to_numpy()

In [27]:
Test_X=Test_X.to_numpy()

Tokenizing the textual data using tfidf vectorizer

In [28]:
Tfidf_vect = TfidfVectorizer()
Tfidf_vect.fit(Train_X)
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

Naive Bayes Classifier

In [29]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  20.166666666666664


Support Vector Machine Classifier (Linear SVM)

In [23]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  55.08333333333333


Predicting the Browse_Node_Id of the test dataset provided by Amazon

In [30]:
test.head()

,PRODUCT_ID,TITLE,DESCRIPTION,BULLET_POINTS,BRAND
0,1,"Command 3M Small Kitchen Hooks, White, Decorat...",Sale Unit: PACK,[INCLUDES - 9 hooks and 12 small indoor strips...,Command
1,2,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...,Synthetic leather palm with double-layer thumb...,[Silicone printing for a better grip. Long las...,O'Neal
2,3,"NFL Detroit Lions Portable Party Fridge, 15.8 ...",Boelter Brands lets you celebrate your favorit...,[Runs on 12 Volt DC Power or 110 Volt AC Power...,Boelter Brands
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...,Features: 50 Station Phonebook Corded Phone Al...,Panasonic Landline Phones doesn't come with a ...,Panasonic
4,5,Zero Baby Girl's 100% Cotton Innerwear Bloomer...,"Zero Baby Girl Panties Set. 100% Cotton, Breat...","[Zero Baby Girl Panties, Pack of 6, 100% Cotto...",Zero


In [31]:
final_test = test[['PRODUCT_ID','TITLE']].copy()

In [32]:
final_test.head()

,PRODUCT_ID,TITLE
0,1,"Command 3M Small Kitchen Hooks, White, Decorat..."
1,2,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...
2,3,"NFL Detroit Lions Portable Party Fridge, 15.8 ..."
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...
4,5,Zero Baby Girl's 100% Cotton Innerwear Bloomer...


In [33]:
final_test.fillna(value= 'the', inplace = True)

In [34]:
final_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97793 entries, 0 to 110774
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PRODUCT_ID  97793 non-null  int64 
 1   TITLE       97793 non-null  object
dtypes: int64(1), object(1)
memory usage: 2.2+ MB


In [35]:
final_test['TITLE'] = final_test['TITLE'].str.lower()

In [36]:
final_test_X = final_test['TITLE']

In [32]:
final_test_X

0         command 3m small kitchen hooks, white, decorat...
1         o'neal jump hardware jag unisex-adult glove (b...
2         nfl detroit lions portable party fridge, 15.8 ...
3         panasonic single line kx-ts880mx corded phone ...
4         zero baby girl's 100% cotton innerwear bloomer...
                                ...                        
110770    aahna e mall oneblade hybrid trimmer shaver an...
110771    grin health n99 anti pollution reusable washab...
110772    asian army pink ultra reusable respirator clot...
110773    im safe 3 ply non-woven disposable surgical fa...
110774    artifie 100% cotton unisex reusable breathable...
Name: TITLE, Length: 97793, dtype: object

In [37]:
Final_Test_X_Tfidf = Tfidf_vect.transform(final_test_X)

Using Naive Bayes rather than SVM cause I do not have enough processing power

In [38]:
final_predictions_NB = Naive.predict(Final_Test_X_Tfidf)

In [39]:
final_predictions_NB

array([1045, 1045, 1045, ..., 1045, 1045,    5])

In [40]:
product_id = final_test[['PRODUCT_ID']].to_numpy()

In [41]:
product_id

array([[     1],
       [     2],
       [     3],
       ...,
       [110773],
       [110774],
       [110775]])

In [42]:
df1 = pd.DataFrame(data = product_id,
                  columns = ['PRODUCT_ID'])

In [43]:
df1

,PRODUCT_ID
0,1
1,2
2,3
3,4
4,5
...,...
97788,110771
97789,110772
97790,110773
97791,110774


In [44]:
df2 = pd.DataFrame(data = final_predictions_NB,
                  columns = ['BROWSE_NODE_ID'])

In [45]:
df2

,BROWSE_NODE_ID
0,1045
1,1045
2,1045
3,1045
4,1045
...,...
97788,1045
97789,1045
97790,1045
97791,1045


In [46]:
df2.describe()

,BROWSE_NODE_ID
count,97793.000000
mean,1000.956398
std,210.466257
min,5.000000
25%,1045.000000
50%,1045.000000
75%,1045.000000
max,4183.000000


In [47]:
train['BROWSE_NODE_ID'].describe()

count    2.110586e+06
mean     9.200541e+03
std      5.520116e+04
min      0.000000e+00
25%      7.390000e+02
50%      1.072000e+03
75%      2.815000e+03
max      2.893574e+06
Name: BROWSE_NODE_ID, dtype: float64

In [48]:
df1.shape

(97793, 1)

In [49]:
df2.shape

(97793, 1)

Making the dataframe into the specific format for submission

In [50]:
DF = df1.join(df2)

In [51]:
DF.to_csv('Amazon_ML_Submission.csv')

THE END !!